<a href="https://colab.research.google.com/github/HanaRjb/Amino-Acid-Frequency-Encoder/blob/main/Task_Avina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch accelerate
!pip install rank_bm25 nltk PyPDF2
!pip install flask datasets


  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

# مثال استفاده:
pdf_path = "/content/Drugs.pdf"  # مسیر فایل PDF خود را وارد کنید
pdf_text = extract_text_from_pdf(pdf_path)
print("نمونه‌ای از متن استخراج شده:")
print(pdf_text[:500])


نمونه‌ای از متن استخراج شده:
 دیکلوفناک چیست و چه کاربردی دارد؟ 
دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهابزا در بدن را مهار و با ایجاد التهاب  
و درد مقابله می کند. این دارو بهصورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجود است:   دیکلوفناک سدیم با شروع اثر آهسته 
و طول اثر بیشتر و دیکلوفناک پتاسیم با شروع اثر سریع و طول اثر کوتاه. این دارو در اشکال دارویی قرص، کپسول، پماد، شیاف و  
آمپول برای استفاده خوراکی یا موضعی در تمام داروخانهها یافت می شود و بهمحض استفاده، دردهای حاد و


In [ ]:
def split_text_into_passages_by_sentences(text, max_sentences=5, min_length=50):
    # تقسیم متن به جملات با فرض اینکه نقطه (.) جداکننده جملات است.
    sentences = text.split('.')
    passages = []
    temp = ""
    count = 0
    for sentence in sentences:
        sentence = sentence.strip()
        if sentence:
            temp += sentence + ". "
            count += 1
            if count >= max_sentences:
                if len(temp.strip()) >= min_length:
                    passages.append(temp.strip())
                temp = ""
                count = 0
    # اضافه کردن باقی‌مانده جملات در صورت وجود
    if temp.strip() and len(temp.strip()) >= min_length:
        passages.append(temp.strip())
    return passages

# تست عملکرد:
passages = split_text_into_passages_by_sentences(pdf_text)
print("تعداد بخش‌های استخراج شده با روش تقسیم‌بندی جملات:", len(passages))


تعداد بخش‌های استخراج شده با روش تقسیم‌بندی جملات: 118


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# ایجاد ایندکس TF-IDF برای بخش‌ها
vectorizer = TfidfVectorizer(stop_words='english')
passages_tfidf = vectorizer.fit_transform(passages)

def retrieve_passages(query, top_k=5):
    query_vec = vectorizer.transform([query])
    # محاسبه شباهت کسینوسی بین پرسش و بخش‌ها
    cosine_similarities = np.dot(passages_tfidf, query_vec.T).toarray().ravel()
    top_indices = np.argsort(cosine_similarities)[::-1][:top_k]
    return [passages[i] for i in top_indices]

# تست عملکرد بازیابی
sample_query = "What are the main conclusions of the study?"
retrieved_context = retrieve_passages(sample_query)
print("بخش‌های بازیابی شده:")
for passage in retrieved_context:
    print(passage)
    print("------")


بخش‌های بازیابی شده:
شرکتهای تولیدکننده  
• داروسازی پارس  
• داروسازی جالینوس  
• داروسازی اسوه 
• داروسازی تهران دارو  
• داروسازی خوارزمی  
• داروسازی حکیم  
• داروسازی تهران شیمی  
• داروسازی اکتوورکو.
------
از مصرف هرگونه محصول آسپرین که بویی شبیه سرکه تند میدهد خودداری   
کنید. همچنین، تمام داروها را از دسترس کودکان و حیوانات خانگی دور نگه دارید. از دفع دارو در سرویس بهداشتی یا سیستم فاضالب خودداری کنید. اگر تاریخ دارو گذشت یا دیگر به آن نیاز نداشتید در مورد دفع  
آن از داروساز یا سیستم فاضالب شهری سؤال بپرسید. اسامی تجاری آسپرین  
آسپرین با اسامی قرص آسپرین،آسپی کور، آسپرین  ۱۸، آسپرین بایر، آسپرین پالس بایر و بوفرین تولید می شود.
------
سوءمصرف دارو عالئم مختلفی دارد، از جمله: 
• درد سوزشی در گلو یا معده؛  
• گیجی؛  
• تغییرات روانی یا خلقوخو؛  
• غش کردن؛  
• ضعف؛ 
• زنگ زدن گوش؛  
• تب؛ 
• تنفس سریع؛  
• تغییر مقدار ادرار؛  
• تشنج؛ 
• از دست دادن اشتها. شرایط نگهداری آسپرین  
آسپرین را در دمای اتاق دور از حرارت و نور، دمای بین ۵۱ تا ۰۳ درجه سانتیگراد نگه دارید. ممکن است برن

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# نام مدل را طبق لینک مورد نظر تنظیم می‌کنیم.
model_name = "unsloth/llama-3-8b-bnb-4bit"  # توجه: ممکن است نیاز به تنظیمات اضافی جهت استفاده از نسخه 4bit داشته باشید

# بارگذاری tokenizer و مدل
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True)

# ایجاد pipeline تولید متن
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=512)

def generate_answer(question, retrieved_context):
    # ترکیب اسناد بازیابی شده به عنوان کانتکست
    context = "\n".join(retrieved_context)
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"
    # تولید پاسخ توسط مدل
    generated = generator(prompt, max_length=200, num_return_sequences=1)
    answer = generated[0]['generated_text']
    return answer

# تست تولید پاسخ
generated_answer = generate_answer(sample_query, retrieved_context)
print("پاسخ تولید شده:")
print(generated_answer)


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "meta-llama/Llama-2-7b-chat-hf"  # نام مدل را بررسی کنید

# بررسی اینکه GPU موجود است یا خیر
device = "cuda" if torch.cuda.is_available() else "cpu"

# پیکربندی quantization با 4bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # فعال‌سازی 4bit
    bnb_4bit_compute_dtype=torch.float16,  # استفاده از float16 برای کاهش مصرف رم
    bnb_4bit_use_double_quant=True,  # استفاده از double quantization برای بهینه‌سازی
)

# بارگذاری توکنایزر
tokenizer = AutoTokenizer.from_pretrained(model_name)

# بارگذاری مدل با quantization روی GPU
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # استفاده از تنظیمات quantization
    device_map="auto"  # تخصیص خودکار مدل به GPU
)

print(f"✅ مدل با موفقیت روی {device.upper()} بارگذاری شد!")


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
403 Client Error. (Request ID: Root=1-67d9dc24-167cc1251a6e1b2b2c571776;463f5a43-1353-4da6-8573-f2354cf2f6bc)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-2-7b-chat-hf to ask for access.

In [ ]:
from google.colab import userdata
hf_token = userdata.get('my-llama-access')  # نام سکرت را همان HF_TOKEN بگذارید

In [ ]:
from huggingface_hub import login
login(token=hf_token)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
!pip install -q -U bitsandbytes

In [ ]:
!pip install -q -U transformers

In [ ]:
pip install -U bitsandbytes


In [ ]:
pip install -U transformers accelerate torch


In [ ]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text


In [ ]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def build_bm25_index(documents):
    tokenized_docs = [word_tokenize(doc.lower()) for doc in documents]
    bm25 = BM25Okapi(tokenized_docs)
    return bm25, tokenized_docs

def retrieve_top_docs(query, bm25, tokenized_docs, top_k=5):
    tokenized_query = word_tokenize(query.lower())
    scores = bm25.get_scores(tokenized_query)
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
    return [documents[i] for i in top_indices]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b-bnb-4bit")
model = AutoModelForCausalLM.from_pretrained("unsloth/llama-3-8b-bnb-4bit", load_in_4bit=True)

def generate_answer(context, query):
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:195: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

In [ ]:
!pip install bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.1 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, render_template_string
from flask_ngrok import run_with_ngrok  # برای اجرا در Colab

app = Flask(__name__)
run_with_ngrok(app)  # فعال‌سازی Ngrok برای اجرای Flask در Colab

def retrieve_top_docs(query):
    """تابع شبیه‌سازی شده برای بازیابی مستندات مرتبط"""
    return [f"مستند مرتبط 1 با {query}", f"مستند مرتبط 2 با {query}"]

def generate_answer(context, query):
    """تابع شبیه‌سازی شده برای تولید پاسخ"""
    return f"پاسخ تولید شده برای '{query}' بر اساس '{context}'"

@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        query = request.form["query"]
        top_docs = retrieve_top_docs(query)
        context = " ".join(top_docs)
        answer = generate_answer(context, query)
        return render_template_string(HTML_TEMPLATE, query=query, answer=answer)
    return render_template_string(HTML_TEMPLATE, query="", answer="")

HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="fa">
<head>
    <meta charset="UTF-8">
    <title>سیستم پرسش و پاسخ</title>
</head>
<body>
    <h2>سیستم پرسش و پاسخ مبتنی بر RAG</h2>
    <form method="post">
        <label>سوال خود را وارد کنید:</label>
        <input type="text" name="query">
        <button type="submit">ارسال</button>
    </form>
    {% if answer %}
        <h3>پاسخ:</h3>
        <p>{{ answer }}</p>
    {% endif %}
</body>
</html>
"""

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           

In [ ]:
!pip install pyngrok


In [ ]:
!pip install streamlit

import streamlit as st

def retrieve_top_docs(query):
    return [f"مستند مرتبط با {query}"]

def generate_answer(context, query):
    return f"پاسخ تولید شده برای '{query}'"

st.title("پرسش و پاسخ")
query = st.text_input("سوال خود را بنویسید:")

if st.button("ارسال"):
    context = " ".join(retrieve_top_docs(query))
    answer = generate_answer(context, query)
    st.write(f"**پاسخ:** {answer}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.2 MB/s eta 0:00:00


2025-03-18 22:52:37.448 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 22:52:37.626 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-18 22:52:37.629 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 22:52:37.632 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 22:52:37.634 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 22:52:37.636 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 22:52:37.638 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 22:52:37.640 Session state does not 

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




⠙⠹⠸⠼⠴⠦⠧⠇⠏
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.240.199.193:8501

your url is: https://crazy-cities-lead.loca.lt
  Stopping...
^C


In [ ]:
%%writefile app.py
import streamlit as st

def retrieve_top_docs(query):
    return [f"مستند مرتبط با {query}"]

def generate_answer(context, query):
    return f"پاسخ تولید شده برای '{query}'"

st.title("پرسش و پاسخ")
query = st.text_input("سوال خود را بنویسید:")

if st.button("ارسال"):
    context = " ".join(retrieve_top_docs(query))
    answer = generate_answer(context, query)
    st.write(f"**پاسخ:** {answer}")


Writing app.py


In [ ]:
!pip install streamlit
!npm install -g localtunnel
